In [2]:
!pip install pyTelegramBotAPI
!install library from pip running this line
!pip install schedule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 5.0 MB/s eta 0:00:00
install: target 'line' is not a directory


In [3]:
import telebot
from datetime import datetime
import json

from telebot import types
import requests
import time
import schedule
import threading

In [ ]:
""" Получим и сохраним токен для нашего бота. """
TOKEN = "6537833236:AAFTpysxmSw03Bz0Rg7cRfWSUseFKcHV3nA"
bot = telebot.TeleBot(TOKEN)
WEATHER_TOKEN = "affc526bdb9959a34cffa24960510d44"
GIPHY_TOKEN = "FS3DtsNb3EYY8yKkFehTsMH1El3VgoWb"


# --------------------------------------------------------
# Handler-ы для обработки команд
# --------------------------------------------------------
""" Обработчик команды start. """
@bot.message_handler(commands=['start'])
def start_handler(message):
    """ Создаем клавиатуру, где будут все возможные функции. """
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                       resize_keyboard=True)

    btn_help = types.KeyboardButton('/что я')
    btn_reg = types.KeyboardButton('/регистрация')
    btn_edit = types.KeyboardButton('/редактировать профиль')
    btn_get_ready = types.KeyboardButton('/собраться')
    btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

    markup.row(btn_edit, btn_get_ready)
    markup.row(btn_reg, btn_help)
    markup.add(btn_get_sub)

    bot.send_message(message.from_user.id,
                     f"Привет, {message.from_user.first_name}! "
                     f"Давай расскажу, что я умею!"
                     f"\n/что я - можешь посмотреть краткую информацию обо мне"
                     f"\n/регистрация - зарегистрируйся, чтобы я знал по какому"
                     f" городу присылать информацию"
                     f"\n/редактировать профиль - если ты сменил город или "
                     f"хочешь посмореть образы для другого пола - жми!"
                     f"\n/собраться - перейдем к сборам на улицу)"
                     f"\n/подписаться или отписаться - хочешь получать погоду "
                     f"каждые 5 часов? - жми!"
                     f"\n/погода - просто посмотри погоду на данный момент"
                     f"\n/как одеться - скажу погоду и покажу что надеть!"
                     f"\n/главное меню - вернемся назад", reply_markup=markup)


""" Обработчик команды "что_я". """
@bot.message_handler(commands=['что'])
def help_handler(message):
    bot.send_message(message.from_user.id,
                     f"Привет, {message.from_user.first_name}! "
                     "Здесь ты можешь узнать погоду на сегодня и посмотреть, "
                     "что в такую погоду можно надеть, чтобы не замерзнуть или "
                     "чтобы не было жарко!)")


""" Обработчик команды "регистрация". """
@bot.message_handler(commands=['регистрация'])
def registration_handler(message):
    """ Проверяем, нет ли у пользователя уже аккаунта. """
    if check_client_in_db(message.from_user.id):
        bot.send_message(message.from_user.id, "Вы уже зарегистрированы")
    else:
        """ Если пользователя еще нет, то собираем с него пол и город. """
        name(message)


""" Обработчик команды "редактировать профиль". """
@bot.message_handler(commands=['редактировать'])
def link_handler(message):
    """ Перед редактиованием нужно проверить, есть ли что редактировать. """
    if check_client_in_db(message.from_user.id):
        """

        Предоставляем пользователю выбор.
        Так как изменить можно либо город, либо пол

        """
        markup = types.InlineKeyboardMarkup()

        btn_gender = types.InlineKeyboardButton(text="Пол",
                                                callback_data="edit_gend")
        btn_town = types.InlineKeyboardButton(text="Город",
                                              callback_data="edit_town")

        markup.add(btn_gender, btn_town)
        bot.send_message(message.from_user.id, "Что хотите изменить?",
                         reply_markup=markup)
    else:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_help = types.KeyboardButton('/что я')
        btn_reg = types.KeyboardButton('/регистрация')
        btn_edit = types.KeyboardButton('/редактировать профиль')
        btn_get_ready = types.KeyboardButton('/собраться')
        btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

        markup.row(btn_edit, btn_get_ready)
        markup.row(btn_reg, btn_help)
        markup.add(btn_get_sub)

        bot.send_message(message.from_user.id,
                         "Вы еще не зарегистрированы, зарегистрируйтесь)",
                         reply_markup=markup)


""" Обработчик команды "подписаться или отписаться". """
@bot.message_handler(commands=['подписаться'])
def help_handler(message):
    """ Подписаться могут только зарегистрированные пользователи. """
    if check_client_in_db(message.from_user.id):
        change_subscription_by_id(message.from_user.id)
        """

        Чтобы пользователь понимал, подписался он или отписался,
        выводим сообщение.

        """
        if client["subscription"]:
            bot.send_message(message.from_user.id, "Вы подписались!")
        else:
            bot.send_message(message.from_user.id, f"Вы отписались( Будем "
            f"ждать вас снова!")
    else:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_help = types.KeyboardButton('/что я')
        btn_reg = types.KeyboardButton('/регистрация')
        btn_edit = types.KeyboardButton('/редактировать профиль')
        btn_get_ready = types.KeyboardButton('/собраться')
        btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

        markup.row(btn_edit, btn_get_ready)
        markup.row(btn_reg, btn_help)
        markup.add(btn_get_sub)

        bot.send_message(message.from_user.id,
                         "Сначала нужно зарегистрироваться!)",
                         reply_markup=markup)


""" Обработчик команды "собраться". """
@bot.message_handler(commands=['собраться'])
def help_handler(message):
    """

    Так как погода зависит от регистрации, то проверяем ее,
    а потом показываем новую клавиатуру, если все ок.

    """
    if check_client_in_db(message.from_user.id):
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_weather = types.KeyboardButton('/погода')
        btn_clothes = types.KeyboardButton('/как одеться')
        btn_back = types.KeyboardButton('/главное меню')

        markup.row(btn_weather, btn_clothes)
        markup.add(btn_back)


        bot.send_message(message.from_user.id, "Давайте начнем собираться!)",
                         reply_markup=markup)
    else:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_help = types.KeyboardButton('/что я')
        btn_reg = types.KeyboardButton('/регистрация')
        btn_edit = types.KeyboardButton('/редактировать профиль')
        btn_get_ready = types.KeyboardButton('/собраться')
        btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

        markup.row(btn_edit, btn_get_ready)
        markup.row(btn_reg, btn_help)
        markup.add(btn_get_sub)

        bot.send_message(message.from_user.id,
                         "Вы еще не зарегистрированы, зарегистрируйтесь)",
                         reply_markup=markup)


""" Обработчик команды "главное меню". """
@bot.message_handler(commands=['главное'])
def help_handler(message):
    """ Тут просто возращаем прошлую клавиатуру. """
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

    btn_help = types.KeyboardButton('/что я')
    btn_reg = types.KeyboardButton('/регистрация')
    btn_edit = types.KeyboardButton('/редактировать профиль')
    btn_get_ready = types.KeyboardButton('/собраться')
    btn_get_sub = types.KeyboardButton('/подписаться на рассылку')

    markup.row(btn_edit, btn_get_ready)
    markup.row(btn_reg, btn_help)
    markup.add(btn_get_sub)

    bot.send_message(message.from_user.id, "Выберите действие)",
                             reply_markup=markup)


""" Обработчик команды "погода". """
@bot.message_handler(commands=['погода'])
def help_handler(message):
    if check_client_in_db(message.from_user.id):
        """

        Получаем информацию о погоде в нужном городе. Если вернулся пустой
        список, значит произошла какая-то ошибка, вероятнее всего человек
        неверно ввел название города, говорим ему, что город не найден.

        """
        weather = get_town_weather(get_town_id(message.from_user.id))
        if weather == None:
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True,
                                               row_width=2,
                                           resize_keyboard=True)

            btn_help = types.KeyboardButton('/что я')
            btn_reg = types.KeyboardButton('/регистрация')
            btn_edit = types.KeyboardButton('/редактировать профиль')
            btn_get_ready = types.KeyboardButton('/собраться')
            btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

            markup.row(btn_edit, btn_get_ready)
            markup.row(btn_reg, btn_help)
            markup.add(btn_get_sub)

            bot.send_message(message.from_user.id, f"Город не найден,"
            f"отредактируйте профиль, изменив город проживания",
                             reply_markup=markup)
        else:
            """ Выводим красивый текст, если все ок. """
            bot.send_message(message.from_user.id,
                         f"Сейчас в городе {get_town_id(message.from_user.id)}"
                         f" {weather[0]}°C"
                         f", но ощущается как {weather[3]}°C"
                         f"\nНа улице {weather[1]}"
                         f"\nВетер {weather[2]} м/с")

            """ Чтобы сообщение было интереснее - выводим рандомную гифку. """
            giphy(message, weather[1])
    else:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_help = types.KeyboardButton('/что я')
        btn_reg = types.KeyboardButton('/регистрация')
        btn_edit = types.KeyboardButton('/редактировать профиль')
        btn_get_ready = types.KeyboardButton('/собраться')
        btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

        markup.row(btn_edit, btn_get_ready)
        markup.row(btn_reg, btn_help)
        markup.add(btn_get_sub)

        bot.send_message(message.from_user.id,
                         "Вы еще не зарегистрированы, зарегистрируйтесь)",
                         reply_markup=markup)


""" Обработчик команды "как одеться". """
@bot.message_handler(commands=['как'])
def help_handler(message):
    """

    Сначала все также, как в команде "погода", так как одежда от нее зависит.
    Потом просто анализируем данные: какая температура, елси тепло, то присылаем
    соответствующий текст и пример того, как одеться. Также ориентируемся на пол
    пользоваателя, чтобы мужчинам присылать мужские фотки, женщинам - женские.
    Также ориентируемся на ветер, так как иногда информации о градусах
    недостаточно, чтобы не замерзнуть. Весь анализ был проведен на собственном
    опыте. Фотографии выгружаем с гугл диска, так вроде очень удобно.
    По крайней мере удобнее, чем в коллабе создавать каждый раз папку с фотками
    Более удобного способа я не придумала)
    Также заботимся о здоровье пользователя и напоминаем о шапке и шарфе зимой
    и о кремах, очках и кпеках летом!)
    Все описанное выше реализуется в функции send_look, которую мы тут вызваем

    """
    if check_client_in_db(message.from_user.id):
        weather = get_town_weather(get_town_id(message.from_user.id))
        if weather == None:
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True,
                                               row_width=2,
                                           resize_keyboard=True)

            btn_help = types.KeyboardButton('/что я')
            btn_reg = types.KeyboardButton('/регистрация')
            btn_edit = types.KeyboardButton('/редактировать профиль')
            btn_get_ready = types.KeyboardButton('/собраться')
            btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

            markup.row(btn_edit, btn_get_ready)
            markup.row(btn_reg, btn_help)
            markup.add(btn_get_sub)

            bot.send_message(message.from_user.id, f"Город не найден,"
            f"отредактируйте профиль, изменив город проживания",
                             reply_markup=markup)
        else:
            send_look(message, weather)
    else:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2,
                                           resize_keyboard=True)

        btn_help = types.KeyboardButton('/что я')
        btn_reg = types.KeyboardButton('/регистрация')
        btn_edit = types.KeyboardButton('/редактировать профиль')
        btn_get_ready = types.KeyboardButton('/собраться')
        btn_get_sub = types.KeyboardButton('/подписаться или отписаться')

        markup.row(btn_edit, btn_get_ready)
        markup.row(btn_reg, btn_help)
        markup.add(btn_get_sub)

        bot.send_message(message.from_user.id,
                         "Вы еще не зарегистрированы, зарегистрируйтесь)",
                         reply_markup=markup)


# --------------------------------------------------------
# Handler-ы для текстовых изображений
# --------------------------------------------------------
@bot.message_handler(content_types=['text'])
def text_handler(message):
    text = message.text.lower()
    """ Делаем так, чтобы на "привет" запускалась работа. """
    if text == "как дела?":
        bot.send_message(message.chat.id, "У меня отлично, работаю вот. А у вас?")
    elif text == "привет":
        if check_client_in_db(message.from_user.id):
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True,
                                               row_width=2,
                                               resize_keyboard=True)

            btn_weather = types.KeyboardButton('/погода')
            btn_clothes = types.KeyboardButton('/как одеться')
            btn_back = types.KeyboardButton('/главное меню')

            markup.row(btn_weather, btn_clothes)
            markup.add(btn_back)
            bot.send_message(message.chat.id, f"Привет! Собираешься на улицу?) "
            f"Давай посмотрим погоду и что можно надеть!", reply_markup=markup)
        else:
            bot.send_message(message.chat.id, f"Привет! Собираешься на улицу?) "
            f"Давай посмотрим погоду и что можно надеть!"
            f" Быстрее регестрируйся!")
    else:
        bot.send_message(message.chat.id, "Я еще не умею читать такой текст(")



# --------------------------------------------------------
# Handler-ы для обработки изображений
# --------------------------------------------------------
@bot.message_handler(content_types=['photo'])
def photos_handler(message):
    """ Просто поддерживаем пользователя гифкой с темой "cool". """
    giphy(message, "cool!")


# --------------------------------------------------------
# Callback-и для Inline markup
# --------------------------------------------------------
"""

Callback, который позволяет нам обрабатывать нажатия на inline-клавиатуре,
при выборе пола нового пользователя.

"""
@bot.callback_query_handler(func=lambda call: "gender" in call.data)
def gender_callback(call):
    if call.data == "gender_man":
        """ Сохраняем пол пользователя. """
        client["gender"] = "man"
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        """ Дальше нужно узнать город, так что вызываем нужную функцию. """
        gender(call.message)

    elif call.data == "gender_woman":
        client["gender"] = "woman"
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        gender(call.message)


"""

Callback, который позволяет нам обрабатывать нажатия на inline-клавиатуре,
при выборе какую именно информацию пользователь хочет изменить.

"""
@bot.callback_query_handler(func=lambda call: "edit" in call.data)
def gender_callback(call):
    if call.data == "edit_gend":
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        """

        При смене гендера, нам нужно предъявить пользователю новый выбор, но так
        как после выбора пола действия должны закончиться, то нужно сделать
        новый Callback и новую функцию.

        """
        edit_gender(call.message)

    elif call.data == "edit_town":
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        """

        Функция по смене города

        """
        edit_town(call.message)


"""

Callback, который позволяет нам обрабатывать нажатия на inline-клавиатуре,
при выборе пола пользователя, который уже зарегестрирован.

"""
@bot.callback_query_handler(func=lambda call: "genit" in call.data)
def gender_callback(call):
    if call.data == "genit_man":
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        save_gender(call.message, "man")
        """

        Так как при редактировании пола действия после его выбора заканчиваются,
        значит просто обновляем информацию о пользователе.

        """

    elif call.data == "genit_woman":
        bot.edit_message_text(chat_id=call.message.chat.id,
                              message_id=call.message.message_id,
                              text=f"Мы приняли ваш выбор!")
        save_gender(call.message, "woman")

"""

Функция, которая при регистрации создает пользователя и предоставляет выбор пола

"""
def name(message):
    global client
    """ Создаем первичный ключ для пользователя. """
    client = {"id": message.from_user.id}

    markup = types.InlineKeyboardMarkup()

    btn_man = types.InlineKeyboardButton(text="Мужчина",
                                         callback_data="gender_man")
    btn_woman = types.InlineKeyboardButton(text="Женщина",
                                           callback_data="gender_woman")

    markup.add(btn_man, btn_woman)
    bot.send_message(message.from_user.id, "Выберите ваш пол",
                     reply_markup=markup)


""" Функция, которая после записи пола спрашивает город проживания. """
def gender(message):
    bot.send_message(message.chat.id, "В каком городе вы сейчас живете?")
    bot.register_next_step_handler(message, town)


"""

Функция, которая после записи города, сохраняет информацю в общую базу данных и
благодарит пользователя.

"""
def town(message):
    client["town"] = message.text
    save_client_info(message)
    bot.send_message(message.chat.id, "Спасибо, мы сохранили ваши данные.")


""" Функция, которая сохраяет в нашу базу данных пользователя. """
def save_client_info(message):
    with open('data.txt') as f:
        """

        При регистрации первого пользователя возникает ошибка пустого файла,
        обрабатываем ее.

        """
        try:
            data = json.load(f)
        except json.decoder.JSONDecodeError:
            data = []

    """

    Так как бот не только регистрирует пользователя, но и редактирует
    информацию о нем, то чтобы id не дублировался, а был уникальным в базе
    данных, нужно удалять прошлую информацию о пользователе.

    """
    if not check_client_in_db(client['id']):
        data.append(client)
    else:
        for idx, stored_client in enumerate(data):
            if stored_client['id'] == client['id']:
                data.pop(idx)
                data.append(client)
                break

    """ Сохраняем данные в файл. """
    with open('data.txt', "w") as f:
        json.dump(data, f, indent=4)
        print("Данные обновлены")


"""

Функция, которая проверяет, зарегестрирован ли уже пользователь.
Тут просто сравнивает id и если такой есть в списке, значит пользователь
зарегестриован.

"""
def check_client_in_db(id):
    with open('data.txt') as f:
        """

        Также, как и при сохранении файла, есть ситуация, когда файл пустой,
        то есть еще никто не зарегестрировался. Эту ошибку надо обрабатывать,
        в данном случае, если файл пустой, значит пользователь не
        зарегестрирован, значит возращаем False.

        """
        try:
            data = json.load(f)
            for i in data:
                if i["id"] == id:
                    return True
        except json.decoder.JSONDecodeError:
            return False

    return False


"""

Функция, меняющая информацию о гендере уже зарегестрированого пользователя.

"""
def edit_gender(message):
    markup = types.InlineKeyboardMarkup()

    btn_man = types.InlineKeyboardButton(text="Мужчина",
                                         callback_data="genit_man")
    btn_woman = types.InlineKeyboardButton(text="Женщина",
                                           callback_data="genit_woman")

    markup.add(btn_man, btn_woman)
    bot.send_message(message.chat.id, "Выберите ваш пол", reply_markup=markup)


""" Функция, принимающая новый город и передающая на сохранение. """
def edit_town(message):
    bot.send_message(message.chat.id, "В каком городе вы сейчас живете?")
    bot.register_next_step_handler(message, save_town)


""" Функция, сохраняющая город в БД. """
def save_town(message):
    with open('data.txt') as f:
            data = json.load(f)

    for i in range(len(data)):
        if data[i]["id"] == message.from_user.id:
            data[i]["town"] = message.text
            break


    with open('data.txt', "w") as f:
        json.dump(data, f)
        print("город сохранен")

    bot.send_message(message.from_user.id, "Город обновлен")

""" Функция, сохраняющая пол в БД. """
def save_gender(message, gender):
    with open('data.txt') as f:
            data = json.load(f)

    for i in range(len(data)):
        if data[i]["id"] == message.chat.id:
            data[i]["gender"] = gender
            break


    with open('data.txt', "w") as f:
        json.dump(data, f)
        print("пол сохранен")

    bot.send_message(message.from_user.id, "Пол обновлен")


"""

Функция, которая по городу возвращает информацию о погоде, а именно -
температуру в градусах, описание, скорость ветра и как все это ощущется.

"""
def get_town_weather(town):
    """

    Получаем погоду с openweathermap, где заранее создала API.
    Извините, что url не по PEP8, просто при переносе он ругается.
    Почитав документацию, поняла, где какая информация содержиться и возвращаем
    ее. Также обрабатываем исключения - в основном тут конечно ошибка может быть
    из-за неверно введенного города, но на всякий и другие обрабатываем.

    """
    try:
        url = 'https://api.openweathermap.org/data/2.5/weather?lang=ru&q=' + town + ',RU,uk&APPID=affc526bdb9959a34cffa24960510d44'
        weather_data = requests.get(url).json()

        temperature = round(weather_data['main']['temp']) - 273
        feel_like = round(weather_data['main']['feels_like']) - 273
        main = weather_data['weather'][0]['description']
        wind = weather_data['wind']['speed']

        weather_info = [temperature, main, wind, feel_like]

    except KeyError as e:
        print("KeyError:", e)
        weather_info = None
    except Exception as e:
        print("An unexpected error occurred:", e)
        weather_info = None

    return weather_info


""" Функция, отправляющая рандомную гифку по теме погоды. """
def giphy(message, main):
    """

    В процессе создания бота, поняла, что giphy находит гифку не под любое
    описание, поэтому если вдруг ошибка, то ищем по городу, тут уже должен
    всегда находить. Если и город по какой-то причине не нашел, то говорим
    пользователю, что к сожалению в этот раз нет гифки.

    """
    try:
        url = "http://api.giphy.com/v1/gifs/random"
        param = {
            "api_key": GIPHY_TOKEN,
            "tag": main,
            "rating": "g",
            "limit": 1,
            "lang": "ru"
        }

        result = requests.get(url, params=param)
        result_dict = result.json()
        link_origin = result_dict["data"]["images"]["original"]["url"]
        bot.send_animation(message.from_user.id, link_origin)

    except TypeError as e:
        print("TypeError:", e)
        url = "http://api.giphy.com/v1/gifs/random"
        param = {
            "api_key": GIPHY_TOKEN,
            "tag": client['town'],
            "rating": "g",
            "limit": 1,
            "lang": "ru"
        }

        result = requests.get(url, params=param)
        result_dict = result.json()
        link_origin = result_dict["data"]["images"]["original"]["url"]
        bot.send_animation(message.from_user.id, link_origin)
    except Exception as e:
        print("An unexpected error occurred:", e)
        bot.send_message(message.from_user.id,
                           "Подходящая гифка в этот раз не нашлась(")


""" Функция, отправляющая примеры образов под погоду. """
def send_look(message, weather):
    """ Эта функция была раписана выше, в команде "как одеться". """
    if get_gender_id(message.from_user.id) == 'woman':
        if weather[0] < -15:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C\n"
                                 f"Одевайтесь по зимнему - теплая куртка, шуба,"
                                 f" пуховик, обязательно шапку и шраф!"
                                 f" Не болейте!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-8lfQcQpLDUf11zlJr-zGuj2rkrSDxdv")
        elif weather[0] < -5:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C и сильный "
                                 f"ветер.\nНадевайте обязательно шапку, чтобы "
                                 f"не простудиться")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-8EQoiGPHRg5e61SRAT4O8VtPfWRYBUP")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nТеплые зимние деньки, так что могут быть "
                                 f"небольшие порывы ветра, следите за горлом)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-8EQoiGPHRg5e61SRAT4O8VtPfWRYBUP")
        elif weather[0] < 10:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C и сильный "
                                 f"ветер.\nЗащищайте горло и уши, чтобы не "
                                 f"простудиться")
                send_photo(message,
                               "https://drive.google.com/uc?id=1--SWYNgNqVJun6YUxwmSe7qfC8vxWQI7")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nНа улице осень-весна, так что могут быть "
                                 f"небольшие порывы ветра, следите за горлом)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-7aZPELq9wy_NvwLa6dzDbCr-LxLP9qz")
        elif weather[0] < 20:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C, довольно "
                                 f"тепло.\nНо дует сильный ветер( Так, что "
                                 f"оденьтесь потеплее, возьмите с собой свитер!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-74nulTlsrXu6ZbbRRAPLVEPV6JhH3l6")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nНа улице уже очень тепло! Ура!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-8fe0p39bEpdcRTxODSQPF-gjW0Gkx6k")
        else:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C, жарко, "
                                 f"дует сильный ветер, но теплый, так что"
                                 f" одевайтесь по-легче!) Наденьте кепку или "
                                 f"бандану, чтобы волосы не лезли в лицо)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-3iJ-yJBVg38R7nVgVwUb0NHRlKoSOUR")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C. Ветра нет, "
                                 f"очень жарко, берегите кожу, голову и глаза"
                                 f" от солнечный лучей! Это важно")
                send_photo(message,
                               "https://drive.google.com/uc?id=1-9eJzeo_wTlub8fn0Uh1TuODj6Aj2X90")
    else:
        if weather[0] < -15:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C\n"
                                 f"Одевайтесь по зимнему - теплая куртка, шуба,"
                                 f" пуховик, обязательно шапку и шраф!"
                                 f" Не болейте!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1rHTqSDF-zkpQpN_uzxJpYht9GJFAqWn8")
        elif weather[0] < -5:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C и сильный "
                                 f"ветер.\nНадевайте обязательно шапку, чтобы "
                                 f"не простудиться")
                send_photo(message,
                               "https://drive.google.com/uc?id=15nTy0Doe4-zgaSXRY_TonP-I1IRI6Toc")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nТеплые зимние деньки, так что могут быть "
                                 f"небольшие порывы ветра, следите за горлом)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1zzM4YwMMOz5rb4yj_BsCVeQO4O7qGYo7")
        elif weather[0] < 10:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C и сильный "
                                 f"ветер.\nЗащищайте горло и уши, чтобы не "
                                 f"простудиться")
                send_photo(message,
                               "https://drive.google.com/uc?id=1No7T25V9oX5mddL6YshIMSKdycJhX8zz")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nНа улице осень-весна, так что могут быть "
                                 f"небольшие порывы ветра, следите за горлом)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1QfxnhpUhSjqbGAV10J3QqG652i2T7Ezg")
        elif weather[0] < 20:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C, довольно "
                                 f"тепло.\nНо дует сильный ветер( Так, что "
                                 f"оденьтесь потеплее, возьмите с собой"
                                 f" свитер!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1cbdcNVZYzZtrZO6RXU6ONPwniCKqa96w")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C."
                                 f"\nНа улице уже очень тепло! Ура!")
                send_photo(message,
                               "https://drive.google.com/uc?id=1_of2hnC0JmsaSgl8w-0ayt-mF_FILXb7")
        else:
            if weather[2] >= 20:
                bot.send_message(message.from_user.id,
                                 "На улице шторм, лучше вообще не выходить!!!")
            elif weather[2] >= 10:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C, жарко, "
                                 f"дует сильный ветер, но теплый, так что"
                                 f" одевайтесь по-легче!) Но не забывайте, "
                                 f"что солнечный удар можно получить и во "
                                 f"время ветра)")
                send_photo(message,
                               "https://drive.google.com/uc?id=1hjI9z_90iMp9Cy4cDHxSzElabfM1lC__")
            else:
                bot.send_message(message.from_user.id,
                                 f"Сейчас на улице {weather[0]}°C. Ветра нет, "
                                 f"очень жарко, берегите кожу, голову и глаза"
                                 f" от солнечный лучей! Это важно")
                send_photo(message,
                               "https://drive.google.com/uc?id=15tIncYtMksM3Y3ft1jHsCbj_RpCCpa-3")


""" Функция, которая по url берет и отправляет фотку. """
def send_photo(message, photo):
    url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    data = {'chat_id': message.from_user.id, 'photo': photo}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        print("Изображение успешно отправлено.")
    else:
        print(response)


""" Функция, меняющая подписку пользователя. """
def change_subscription_by_id(id_client):
    with open('data.txt') as f:
        data = json.load(f)

    for i in range(len(data)):
        if data[i]["id"] == id_client:
            subscription = data[i].get("subscription")
            if subscription != None:
                data[i]["subscription"] = not subscription
                client["subscription"] = not subscription
            else:
                data[i]["subscription"] = True
                client["subscription"] = True

    with open('data.txt', "w") as f:
        json.dump(data, f)
        print("Подписка обновлена")


""" Функция, которая возвращает город, по id. """
def get_town_id(id):
    with open('data.txt') as f:
        data = json.load(f)
        result_town = ""
        for i in data:
            if i["id"] == id:
                result_town = i["town"]
                break

    return result_town


""" Функция, которая возвращает город, по id. """
def get_gender_id(id):
    with open('data.txt') as f:
        data = json.load(f)
        result_gender = ""
        for i in data:
            if i["id"] == id:
                result_gender = i["gender"]
                break

    return result_gender


""" Функция, возвращающая список подписчиков и их города. """
def get_subscribes_id_list():
    with open('data.txt') as f:
        data = json.load(f)
        result_id = []
        for i in data:
            if i["subscription"]:
                result_id.append((i["id"], i["town"]))

    return result_id


""" Планировщик для отправки сообщений по времени. """
def send_weather_at_8_00(bot):
    """ Функция отправки сообщения. """
    def send_weather():
        for client_id, client_town in get_subscribes_id_list():
            weather = get_town_weather(client_town)
            bot.send_message(client_id,
                         f"Сейчас в городе {client_town} {weather[0]}°C"
                         f", но ощущается как {weather[3]}°C"
                         f"\nНа улице {weather[1]}"
                         f"\nВетер {weather[2]} м/с")
            print("Отправка завершена!")

    schedule.every(5).hours.do(send_weather)

    while True:
        schedule.run_pending()
        time.sleep(1)

thread_scheduler = threading.Thread(target=send_weather_at_8_00, args=(bot,))
""" Запускаем поток. """
thread_scheduler.start()

""" Зацикливаем нашего бота, чтобы он всё обрабатывал. """
bot.polling(none_stop=True, interval=0)

2024-03-28 11:21:50,275 (__init__.py:1147 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bot was blocked by the user"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bot was blocked by the user
2024-03-28 11:21:50,279 (__init__.py:1149 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1141, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 149, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 92, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 7868, in _run_middlewares_and_handler
    r

Данные обновлены
TypeError: list indices must be integers or slices, not str
Изображение успешно отправлено.
Изображение успешно отправлено.
Подписка обновлена
Подписка обновлена
Данные обновлены
Данные обновлены
Данные обновлены
Изображение успешно отправлено.
Изображение успешно отправлено.
Подписка обновлена
Данные обновлены


2024-03-28 11:33:39,929 (__init__.py:1147 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bots can't send messages to bots"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bots can't send messages to bots
2024-03-28 11:33:39,933 (__init__.py:1149 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1141, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 149, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 92, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 7868, in _run_middlewares_and_han

пол сохранен
Изображение успешно отправлено.
Данные обновлены
Изображение успешно отправлено.
